In [8]:
import pandas as pd
df = pd.read_csv('/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
print("shape:", df.shape)
print(df['sentiment'].value_counts())

# A couple of random samples
df.sample(5)

shape: (50000, 2)
sentiment
positive    25000
negative    25000
Name: count, dtype: int64


,review,sentiment
32930,Though it pains me to some degree that I'm bot...,negative
42666,This is really the only chance to see the magi...,positive
11077,This movie was such a blast! It has that feel-...,positive
29728,I watched it with my mom and we were like...<b...,negative
232,"I saw this movie in the theater, and was thoro...",positive


In [10]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [12]:
df.isnull().sum()


,0
review,0
sentiment,0


In [13]:
import re
from bs4 import BeautifulSoup

def clean_text(text):
    # remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # remove non-letters
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)
    # lowercase and normalize spaces
    text = text.lower().strip()
    text = re.sub(r'\s+', ' ', text)
    return text

df['text_clean'] = df['review'].apply(clean_text)
df.head()


,review,sentiment,text_clean
0,One of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,a wonderful little production the filming tech...
2,I thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,basically there s a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei s love in the time of money is a...


In [14]:
df['label'] = df['sentiment'].map({'positive':1, 'negative':0})
df[['text_clean','label']].head()

,text_clean,label
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there s a family where a little boy ...,0
4,petter mattei s love in the time of money is a...,1


In [15]:
from sklearn.model_selection import train_test_split

X = df['text_clean'].values
y = df['label'].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

print(len(X_train), len(X_val))

42500 7500


In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 30000
maxlen = 200
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

# convert texts to sequences
train_seq = tokenizer.texts_to_sequences(X_train)
val_seq = tokenizer.texts_to_sequences(X_val)

# pad
X_train_pad = pad_sequences(train_seq, maxlen=maxlen, padding='post', truncating='post')
X_val_pad   = pad_sequences(val_seq,   maxlen=maxlen, padding='post', truncating='post')

print("X_train_pad shape:", X_train_pad.shape)

X_train_pad shape: (42500, 200)


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, Input

embedding_dim = 50

baseline = Sequential([
    Input(shape=(maxlen,)),                     # Explicit input shape
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    GlobalAveragePooling1D(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

baseline.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
baseline.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 50)        │     1,500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 50)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,501,665 (5.73 MB)

 Trainable params: 1,501,665 (5.73 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('baseline_imdb.h5', save_best_only=True, monitor='val_loss')
]

history_baseline = baseline.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=7,
    batch_size=128,
    callbacks=callbacks
)


Epoch 1/7
330/333 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6409 - loss: 0.6261

333/333 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.6421 - loss: 0.6249 - val_accuracy: 0.8361 - val_loss: 0.3652
Epoch 2/7
331/333 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8650 - loss: 0.3211

333/333 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.8651 - loss: 0.3210 - val_accuracy: 0.8447 - val_loss: 0.3463
Epoch 3/7
332/333 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8985 - loss: 0.2581

333/333 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.8985 - loss: 0.2580 - val_accuracy: 0.8909 - val_loss: 0.2741
Epoch 4/7
333/333 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.9236 - loss: 0.2019 - val_accuracy: 0.8737 - val_loss: 0.3050
Epoch 5/7
333/333 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.9388 - loss: 0.1735 - val_accuracy: 0.8871 - val_loss: 0.2901
Epoch 6/7
333/333 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.9474 - loss: 0.1499 - val_accuracy: 0.8859 - val_loss: 0.2957


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# -----------------------------
# Corrected LSTM Model
# -----------------------------
model_lstm = Sequential([
    Input(shape=(maxlen,)),                           # FIX: define input shape
    Embedding(input_dim=vocab_size, output_dim=200),  # Embedding layer
    Bidirectional(LSTM(64, return_sequences=False)),  # BiLSTM
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_lstm.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_lstm.summary()

# -----------------------------
# Callbacks
# -----------------------------
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('lstm_imdb.h5', save_best_only=True, monitor='val_loss')
]

# -----------------------------
# Training
# -----------------------------
history_lstm = model_lstm.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=8,
    batch_size=128,
    callbacks=callbacks
)



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 200, 200)       │     6,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │       135,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,144,001 (23.44 MB)

 Trainable params: 6,144,001 (23.44 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/8
333/333 ━━━━━━━━━━━━━━━━━━━━ 0s 790ms/step - accuracy: 0.6998 - loss: 0.5395

333/333 ━━━━━━━━━━━━━━━━━━━━ 281s 831ms/step - accuracy: 0.7001 - loss: 0.5392 - val_accuracy: 0.8461 - val_loss: 0.3581
Epoch 2/8
333/333 ━━━━━━━━━━━━━━━━━━━━ 0s 798ms/step - accuracy: 0.8941 - loss: 0.2857

333/333 ━━━━━━━━━━━━━━━━━━━━ 324s 839ms/step - accuracy: 0.8941 - loss: 0.2856 - val_accuracy: 0.8883 - val_loss: 0.2920
Epoch 3/8
333/333 ━━━━━━━━━━━━━━━━━━━━ 322s 839ms/step - accuracy: 0.9450 - loss: 0.1551 - val_accuracy: 0.8780 - val_loss: 0.3140
Epoch 4/8
333/333 ━━━━━━━━━━━━━━━━━━━━ 278s 835ms/step - accuracy: 0.9460 - loss: 0.1505 - val_accuracy: 0.8715 - val_loss: 0.4358
Epoch 5/8
333/333 ━━━━━━━━━━━━━━━━━━━━ 278s 834ms/step - accuracy: 0.9804 - loss: 0.0604 - val_accuracy: 0.8717 - val_loss: 0.4497


In [20]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Predict (use model_lstm or baseline; choose best)
pred_prob = model_lstm.predict(X_val_pad, batch_size=256)
pred = (pred_prob.flatten() >= 0.5).astype(int)

print(classification_report(y_val, pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_val, pred))

30/30 ━━━━━━━━━━━━━━━━━━━━ 13s 440ms/step
              precision    recall  f1-score   support

           0     0.8931    0.8821    0.8876      3750
           1     0.8836    0.8944    0.8889      3750

    accuracy                         0.8883      7500
   macro avg     0.8883    0.8883    0.8883      7500
weighted avg     0.8883    0.8883    0.8883      7500

Confusion matrix:
 [[3308  442]
 [ 396 3354]]


In [21]:
# Save tokenizer
import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Save model already saved by ModelCheckpoint; or explicitly:
model_lstm.save('final_lstm_imdb.h5')


In [22]:
def predict_review(text, model, tokenizer, maxlen=200):
    txt = clean_text(text)
    seq = tokenizer.texts_to_sequences([txt])
    pad = pad_sequences(seq, maxlen=maxlen, padding='post', truncating='post')
    p = model.predict(pad)[0][0]
    return p  # probability [0..1]

# example
example = "this movie was thrilling and the acting was superb"
print("Positive prob:", predict_review(example, model_lstm, tokenizer, maxlen))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Positive prob: 0.8945015


In [23]:
# Your function
def predict_review(text, model, tokenizer, maxlen=200):
    txt = clean_text(text)
    seq = tokenizer.texts_to_sequences([txt])
    pad = pad_sequences(seq, maxlen=maxlen, padding='post', truncating='post')
    p = model.predict(pad)[0][0]
    return p  # probability [0..1]

# List of reviews
reviews = [
    "The storyline was gripping and kept me engaged the entire time.",
    "Great visuals, but the plot felt a little predictable.",
    "Unexpectedly amazing movie with strong character development.",
    "Some scenes were slow, but the emotional moments were powerful.",
    "A perfect blend of comedy and drama with an outstanding lead performance.",
    "One of the best movies I’ve seen this year with excellent suspense.",
]

# Loop to check sentiment of each review
for i, review in enumerate(reviews, start=1):
    prob = predict_review(review, model_lstm, tokenizer, maxlen)
    sentiment = "Positive" if prob >= 0.5 else "Negative"
    print(f"Review {i}: {sentiment} ({prob:.4f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Review 1: Positive (0.7074)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Review 2: Positive (0.7222)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Review 3: Positive (0.9208)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Review 4: Positive (0.6013)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Review 5: Positive (0.9102)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Review 6: Positive (0.9543)


In [30]:
# Your existing function
def predict_review(text, model, tokenizer, maxlen=200):
    txt = clean_text(text)
    seq = tokenizer.texts_to_sequences([txt])
    pad = pad_sequences(seq, maxlen=maxlen, padding='post', truncating='post')
    p = model.predict(pad)[0][0]
    return p  # probability [0..1]

# Example: single prediction
example = "this movie was thrilling and the acting was superb"
print("Positive prob:", predict_review(example, model_lstm, tokenizer, maxlen))

# -----------------------------
# New Reviews to Test
# -----------------------------
reviews = [
    "The movie delivers a beautifully layered narrative that unfolds with emotional depth, supported by performances that feel genuinely heartfelt.",
    "A stunning blend of visual elegance and thoughtful storytelling, this film manages to impress both technically and emotionally.",
    "The director’s vision shines through every frame, creating a cinematic experience that feels both intimate and grand in scale.",
    "This movie balances character development and plot progression so well that every scene feels meaningful and carefully crafted.",
    "A captivating story enriched with strong acting and atmospheric cinematography, making it far more memorable than expected.",
    "With its thoughtful script and remarkable performances, the film succeeds in delivering a powerful emotional journey.",
    "The pacing is deliberate yet engaging, slowly drawing the audience into a world filled with complex emotions and compelling surprises.",
    "The chemistry between the cast elevates the entire narrative, giving even the simplest moments a sense of authenticity.",
    "The film’s visual storytelling is truly impressive, using subtle details to enhance the emotional weight of the story.",
    "Despite exploring familiar themes, the movie approaches them with such nuance that everything feels fresh and meaningful.",
    "An elegantly crafted film that balances emotion and spectacle, leaving a long-lasting impression.",
    "The soundtrack perfectly complements the narrative, enhancing the emotional resonance of every critical scene.",
    "The dialogue is crisp and engaging, adding depth to characters and making their relationships feel natural and impactful.",
    "A deeply satisfying watch, thanks to its careful character buildup and beautifully executed emotional arcs.",
    "The film’s ability to maintain tension while still delivering heartfelt moments is genuinely admirable.",
    "It combines stunning visuals with thoughtful themes, creating a layered experience that rewards attentive viewers.",
    "Every character feels purposeful, and their individual journeys intertwine seamlessly into a cohesive narrative.",
    "The movie takes its time developing its world, resulting in a rich and immersive cinematic atmosphere.",
    "An impressive film with emotional depth, strong performances, and a story that unfolds with grace and clarity.",
    "The blend of humor and drama is handled masterfully, making the movie both entertaining and thought-provoking.",
    "Its narrative sophistication makes it more than just a typical film—it’s a genuine piece of art.",
    "The lead actor delivers a nuanced performance that anchors the story with extraordinary emotional realism.",
    "A film that successfully balances subtlety and impact, leaving the audience with plenty to reflect upon.",
    "The cinematography is breathtaking, capturing the tone and emotion of the film with incredible precision.",
    "A compelling story enriched with heartfelt moments and beautifully written character arcs.",
    "The direction is sharp and focused, ensuring that every scene contributes to the film’s central themes.",
    "This movie showcases how powerful storytelling can elevate even a simple plot into something unforgettable.",
    "Every emotional moment lands perfectly, thanks to the brilliant synergy between the cast and the script.",
    "The film’s buildup is steady and rewarding, offering a satisfying payoff that resonates long after it ends.",
    "The subtle use of symbolism adds depth to the narrative without overwhelming the viewer.",
    "An engaging blend of charm and emotional weight, supported by confident and expressive performances.",
    "From visuals to dialogue, every aspect of the film feels carefully refined and artistically intentional.",
    "The story unfolds with a graceful balance of melancholy and hope, making the film deeply moving.",
    "The attention to detail in both production and storytelling elevates this movie far above standard cinema.",
    "Each scene carries emotional purpose, contributing to a beautifully cohesive and satisfying narrative.",
    "The film maintains a warm emotional tone while still exploring complex and mature themes.",
    "A truly compelling experience that invites viewers to connect deeply with its characters.",
    "The narrative structure is clever and well-paced, ensuring that interest never fades for a moment.",
    "The movie blends its visual and emotional elements so effectively that it feels effortlessly immersive.",
    "An extraordinary film that balances artistic ambition with accessible emotional storytelling.",
    "The characters evolve naturally, making their personal journeys feel authentic and rewarding.",
    "It combines strong thematic elements with gorgeous cinematography, resulting in a remarkably polished film.",
    "The emotional highs and lows are handled with impeccable sensitivity, creating a truly heartfelt experience.",
    "This film proves how powerful subtle acting and careful writing can be when executed with precision.",
    "Every moment feels intentional, and the narrative carries a sense of purpose that strengthens the overall impact.",
    "The storytelling is rich with detail and emotional nuance, making the viewing experience both engaging and meaningful.",
    "The film’s blend of realism and artistry gives it a unique charm that’s hard to forget.",
    "A beautifully balanced film that manages to entertain while also providing thoughtful commentary on its themes.",
    "The cast’s chemistry is undeniable, elevating even the quieter scenes into emotionally rich moments.",
    "A rare movie that manages to be visually stunning, emotionally powerful, and narratively engaging all at once.",
    #Negative
    "Despite its ambitious premise, the movie collapses under the weight of its poorly structured narrative and inconsistent character motivations.",
    "The film attempts to deliver emotional depth, but its shallow dialogue and forced sentimentality make every dramatic moment feel artificial.",
    "A visually dull production with no creative spark; the storyline meanders aimlessly without offering meaningful progression.",
    "What could have been an engaging exploration of the theme turns into a cluttered mess filled with unnecessary subplots and weak resolutions.",
    "The pacing is disastrously uneven, shifting abruptly from slow, dragging scenes to rushed sequences that barely make sense.",
    "Despite a few promising scenes, the overall movie lacks coherence, as if the writers stitched together disconnected ideas.",
    "The emotional beats fail entirely, mostly because the characters are too underdeveloped to evoke any real investment from the audience.",
    "An overconfident film that mistakes loud music and flashy visuals for genuine storytelling and emotional impact.",
    "The performances suffer from poor direction, resulting in characters that feel robotic and devoid of authenticity.",
    "Although the film tries to appear profound, it ultimately presents superficial commentary with no real substance.",
    "The story is plagued by predictable twists that offer no surprise, making the entire journey painfully uninspiring.",
    "Even with its long runtime, the film fails to explain crucial plot points, leaving several unresolved threads dangling pointlessly.",
    "The screenplay is overloaded with exposition, as if the filmmakers had no trust in the audience’s ability to understand subtle storytelling.",
    "Instead of building tension naturally, the movie relies on repetitive shock tactics that quickly lose their effectiveness.",
    "The visuals could have carried the film, but the uninspired cinematography makes it feel like a low-effort production.",
    "The protagonist’s arc is confusing and inconsistent, making it difficult to empathize with or even understand their decisions.",
    "A film filled with potential but ruined by monotonous dialogue and an overly simplistic treatment of complex themes.",
    "The attempt at humor feels painfully outdated, and many jokes fall flat due to poor timing and weak writing.",
    "The movie leans too heavily on nostalgia, offering nothing original or memorable to justify its existence.",
    "The constant tonal shifts—from comedy to drama to action—create a disjointed experience that never finds its identity.",
    "The soundtrack is overused and emotionally manipulative, attempting to force feelings that the script never earns.",
    "The editing is chaotic, making it difficult to follow the storyline as scenes cut abruptly without proper transition.",
    "The antagonists are so poorly written that they feel like cardboard obstacles rather than legitimate threats or characters.",
    "The world-building lacks detail and consistency, leaving the setting feeling incomplete and unbelievable.",
    "Far too much time is spent on filler scenes, while important story developments are glossed over or rushed.",
    "The film desperately tries to seem complex, but its philosophical themes are shallow and poorly executed.",
    "Even the action scenes, which should have been thrilling, are dull and lack any sense of urgency or creativity.",
    "The romantic subplot feels unnecessary and distracts from the main storyline without adding emotional value.",
    "Characters make irrational decisions solely to push the plot forward, making the story feel contrived and unnatural.",
    "The climax is underwhelming and leaves the audience feeling empty rather than satisfied or impressed.",
    "The film’s numerous clichés make it feel like an uninspired rehash of better movies that came before it.",
    "Despite the heavy dramatic tone, the movie fails to evoke emotion because it never allows meaningful character exploration.",
    "The central conflict is introduced late and resolved too quickly, creating an imbalance that weakens the narrative arc.",
    "Attempts at suspense fall flat due to predictable setups and an overreliance on generic tension-building tropes.",
    "The dialogue is cluttered with unnecessary monologues that add length but no real insight or depth.",
    "Even the supporting cast feels wasted as their characters are given little purpose or development.",
    "The story lacks originality and seems patched together from random ideas that never fully connect.",
    "With its repetitive scenes, the movie feels much longer than it actually is, making it a tedious experience.",
    "Despite being marketed as an emotional journey, the film fails to deliver any genuine heartfelt moments.",
    "The cinematography fails to enhance the story, instead making scenes feel flat and visually monotonous.",
    "Every attempt at creating mystery is ruined by predictable reveals and obvious foreshadowing.",
    "The movie spends too much time explaining its themes instead of showing them through meaningful storytelling.",
    "The villain’s motivations are so poorly explained that the conflict comes across as trivial and unconvincing.",
    "Even with a talented cast, the film feels lifeless due to uninspired writing and weak direction.",
    "The narrative tone feels condescending, as if the film underestimates the audience’s intelligence.",
    "The emotional climax feels unearned because the film never invests enough time developing the relationships between characters.",
    "The ending is abrupt and offers no closure, leaving the entire experience feeling incomplete and unsatisfying.",
    "Instead of building tension organically, the movie relies on loud noises and sudden cuts that quickly become irritating.",
    "Despite a promising setup, the second half of the film completely collapses into rushed storytelling and cheap resolutions.",
    "The film ultimately feels like a wasted opportunity—a shallow, poorly constructed narrative that fails to engage on any meaningful level."]

# -----------------------------
# Predict Sentiment for Each
# -----------------------------
print("\nSentiment Predictions:\n")
for r in reviews:
    prob = predict_review(r, model_lstm, tokenizer, maxlen)
    label = "Positive" if prob >= 0.5 else "Negative"
    print(f"{label} ({prob:.4f}) -> {r}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Positive prob: 0.8945015

Sentiment Predictions:

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Positive (0.9787) -> The movie delivers a beautifully layered narrative that unfolds with emotional depth, supported by performances that feel genuinely heartfelt.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Positive (0.9262) -> A stunning blend of visual elegance and thoughtful storytelling, this film manages to impress both technically and emotionally.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Positive (0.6314) -> The director’s vision shines through every frame, creating a cinematic experience that feels both intimate and grand in scale.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Negative (0.3166) -> This movie balances character development and plot progression so well that every scene feels meaningful and carefully crafted.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Positive (0.8883) -> A captivating story enriched with strong acting and atmospheric cinematography, making it f